# DeepDTA(논문을 참고한 Data 처리부터 모델 구현)
> 2022-10-18

- toc:true
- branch: master
- badges: true
- comments: false
- author: blog_owner


`-` 이전의 블로그에서 가져왔다.

`-` 이번 모델 구현은 DeepDTA를 기반으로 하긴 하였지만, 논문 참고만 하고 정답같은 것 없이 데이터 처리, 모델 등을 구현한 것이라 DeepDTA 구현 모델이라고 하긴 힘들 것 같다.

`-` 모델을 만드는데 사용한 라이브러리는 pytorch이다.

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split


import numpy as np

import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

from tqdm import tqdm

/home/neogreen/anaconda3/envs/chihyeon/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
if torch.cuda.is_available():
    print('use GPU')
    device = 'cuda'
else:
    print('use CPU')
    device = 'cpu'

use GPU


## Data processing

## Drug

`-` 데이터 읽기

In [6]:
with open("drug.txt","r") as f:
    lines = f.readlines()

`-` SMILES만 남도록 데이터 처리

In [7]:
drug = list(str(lines).split())
drugs = []
for i in range(len(drug)//2):
    drugs.append(drug[2*i+1])


In [11]:
__drugs = list(map(lambda s: s[1:-2], drugs))

In [12]:
__drugs[-1] = __drugs[-1][:-4]

`-` drug를 데이터 프레임 형태로 저장

In [14]:
drugs = __drugs
_drug = pd.DataFrame({"drugs":drugs})

## Protein

In [15]:
with open("proteins.txt","r") as f:
    lines = f.readlines()

In [16]:
protein = list(str(lines).split())
proteins = []
for i in range(len(protein)//2):
    proteins.append(protein[2*i+1])


In [19]:
__proteins = list(map(lambda s: s[1:-2], proteins))

In [21]:
__proteins[-1] = __proteins[-1][:-2]

In [23]:
proteins = __proteins

`-` sequence안의 각 amino acid character를 모아서 vocabulary 생성

In [24]:
set_proteins = ''.join(map(str, proteins)) + "@" # "@" for padding or End
proteins_vocab = set(set_proteins)
proteins_vocab_size = len(set(set_proteins))

`-` protein 데이터를 Dataframe 형태로

In [25]:
_protein =  pd.DataFrame({"proteins":proteins})

## Drug와 Protein 데이터를 하나의 Data frame에 묶음

`-` 최종적으로 사용할 데이터는 Drug와 Protein간의 affinity이다.  
데이터는 Drug 1과 Protein 1의 affinity, Drug 1과 Protein 2의 affinity, Drug 1과 Protein 3의 affinity, ... , Drug m과 Protein n의 affinity 와같은 형태로 각 drug마다 모든 protein에 대한 affinity 형태로 데이터가 주어져있다.

따라서 둘을 합칠 때는 Drug 1, Drug 2, ... Drug m 각각 에 대해서 Broadcasting을 하는 방식으로 protein과 데이터를 합쳐야한다.

In [26]:
data = _drug.merge(_protein, how='cross')

- 위와같은 broadcasting을 하면서 병합하는 방법이 merge안의 인자에 how='cross'를 하는 것

`-` 이제 각 drug와 protein의 affinity 값들을 불러와서 위에서 만든 drug-protein pair마다 해당 값들을 넣어야 한다.

In [27]:
with open("drug_protein.txt","r") as f:
    lines = f.readlines()

In [28]:
lines = list(map(lambda x: x[:-1], lines))
lines = list(map(lambda x: x.split(), lines))


`-` 이걸 이제, 각각의 요소를 str에서 float 타입으로 바꿔야하고, 하나 벡터로 flatten해서 펼쳐서 그대로 dataframe에 affinity를 ㅜ이한 column하나를 추가해서 붙여야한다.

In [29]:
for i in range(len(lines)):
    lines[i] = list(map(lambda x: float(x), lines[i]))

In [30]:
drug_protein_affinity = torch.tensor(lines)

In [31]:
drug_protein = drug_protein_affinity.reshape([-1]) # 이게 잘 flatten 됬는지 확인하려면 drug_protein_affinity[0]이랑 drug_protein 의 첫번째 drug의 모든 protein에 대한 affinitiy인 442개의 첫번째 요소들이 같은지 보면 됨

`-` 논문에서는 affinity를 그냥 사용하지 않고 10^9로 나누고 log를 적용한 값을 사용하였다.

In [32]:
affinity = -torch.log10(drug_protein/(10**9))

- 논문에서 affinity 그대로 안쓰고 계산과정 한번 거침

In [33]:
data["affinity"] = affinity

`-` 중복되는 데이터 제거

In [34]:
data = data.drop_duplicates()

In [35]:
data = data.loc[data[["drugs","proteins"]].drop_duplicates().index]

`-` 이전에 merge를 할 때 broadcast방식으로 해서 데이터의 순서가 Drug 1과 Protein 1의 affinity, Drug 1과 Protein 2의 affinity, Drug 1과 Protein 3의 affinity 이런식으로 되었다.  
이 상태로 training[0:0.5], test[0.5:1] 로 나눠버리고 학습을 진행한다면 0.5 이후의 Drug Smiles는 training동안에 못본 데이터여서 성능이 잘 안나올 것이다. 

따라서 아래와 같은 코드로 shuffle을 한다.

In [36]:
data = data.sample(frac=1)
data = data.sample(frac=1)
data

,drugs,proteins,affinity
10519,C1CC(=NO)C2=C1C=C(C=C2)C3=CN(N=C3C4=CC=NC=C4)CCO,MPARIGYYEIDRTIGKGNFAVVKRATHLVTKAKVAIKIIDKTQLDE...,5.000000
7103,CC(C1=C(C=CC(=C1Cl)F)Cl)OC2=C(N=CC(=C2)C3=CN(N...,MEGAAAPVAGDRPDLGLGAPGSPREAVAGATAALEPRKPHGVKRHH...,5.000000
16942,CCOC1=C(C=C2C(=C1)N=CC(=C2NC3=CC(=C(C=C3)OCC4=...,MCTVVDPRIVRRYLLRRQLGQGAYGIVWKAVDRRTGEVVAIKKIFD...,5.000000
7334,CC(C1=C(C=CC(=C1Cl)F)Cl)OC2=C(N=CC(=C2)C3=CN(N...,MAHSPVAVQVPGMQNNIADPEELFTKLERIGKGSFGEVFKGIDNRT...,5.552842
21242,CN(C)CC1CCN2C=C(C3=CC=CC=C32)C4=C(C5=CN(CCO1)C...,MSDVTIVKEGWVQKRGEYIKNWRPRYFLLKTDGSFIGYKEKPQDVD...,5.853872
...,...,...,...
19552,CC(C)N1C2=C(C(=C3C=C4C=C(C=CC4=N3)O)N1)C(=NC=N2)N,MACLHETRTPSPSFGGFVSTLSEASMRKLDPDTSDCTPEKDLTPTH...,5.000000
12017,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,MRHSKRTHCPDWDSRESWGHESYRGSHKRKRRSHSSTQENRHCKPH...,5.677781
13284,CS(=O)(=O)CCNCC1=CC=C(O1)C2=CC3=C(C=C2)N=CN=C3...,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,5.000000
1908,CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(...,MKPATGLWVWVSLLVAAGTVQPSDSQSVCAGTENKLSSLSDLEQQY...,8.200660


- 같은 SMILES가 연속으로 나오지 않는 것을 통해 Shuffle이 잘 되었음을 확인하였다.

`-` char를 int로 바꾸는 작업 + max length에 미치지 못하는 sequnece는 @에 대응되는 integer로 추가로 padding

In [37]:
def word2int(x, length):
    set_x = ''.join(map(str, x)) + "@"
    
#    print(x[0])

    set_x = set_x + "@" # "@" for padding or End
    x_vocab = set(set_x)
    
    x_word_dict = {w: i for i, w in enumerate(x_vocab)}
    x_number_dict = {i: w for i, w in enumerate(x_vocab)}
    
#    print(x_word_dict)
    
    len_x = list(map(lambda s: len(s), x))
    max_len_x = max(len_x)
    
    
    x_mapped = []
    for i in x:
        one_molecule = []
        for j in i:
            if(len(one_molecule)<length):
                try:
                    one_molecule.append(x_word_dict[j])
                except:
                    pass

        while(len(one_molecule)<length):  # 논문에서 85로 제한함
            one_molecule.append(x_word_dict["@"])

        x_mapped.append(one_molecule)
        
    

    return x_mapped, len(x_vocab)

`-` drug와 protein에 대해서 각각 character를 dictionary의 대응되는 integer로 바꾸고 설정한 max 길이만큼 모자라면 @로 패딩한다.

In [38]:
_drugs, _drug_vocab = word2int(data["drugs"].values,85)
_proteins, _protein_vocab = word2int(data["proteins"].values,1200)



C1CC(=NO)C2=C1C=C(C=C2)C3=CN(N=C3C4=CC=NC=C4)CCO
{'O': 0, '7': 1, 'P': 2, 'C': 3, 'F': 4, 'l': 5, '6': 6, '8': 7, '(': 8, 'B': 9, '2': 10, 'I': 11, '5': 12, '9': 13, 'S': 14, '=': 15, 'N': 16, '#': 17, '.': 18, ')': 19, '4': 20, '@': 21, 'r': 22, '1': 23, '3': 24}
MPARIGYYEIDRTIGKGNFAVVKRATHLVTKAKVAIKIIDKTQLDEENLKKIFREVQIMKMLCHPHIIRLYQVMETERMIYLVTEYASGGEIFDHLVAHGRMAEKEARRKFKQIVTAVYFCHCRNIVHRDLKAENLLLDANLNIKIADFGFSNLFTPGQLLKTWCGSPPYAAPELFEGKEYDGPKVDIWSLGVVLYVLVCGALPFDGSTLQNLRARVLSGKFRIPFFMSTECEHLIRHMLVLDPNKRLSMEQICKHKWMKLGDADPNFDRLIAECQQLKEERQVDPLNEDVLLAMEDMGLDKEQTLQSLRSDAYDHYSAIYSLLCDRHKRHKTLRLGALPSMPRALAFQAPVNIQAEQAGTAMNISVPQVQLINPENQIVEPDGTLNLDSDEGEEPSPEALVRYLSMRRHTVGVADPRTEVMEDLQKLLPGFPGVNPQAPFLQVAPNVNFMHNLLPMQNLQPTGQLEYKEQSLLQPPTLQLLNGMGPLGRRASDGGANIQLHAQQLLKRPRGPSPLVTMTPAVPAVTPVDEESSDGEPDQEAVQSSTYKDSNTLHLPTERFSPVRRFSDGAASIQAFKAHLEKMGNNSSIKQLQQECEQLQKMYGGQIDERTLEKTQQQHMLYQQEQHHQILQQQIQDSICPPQPSPPLQAACENQPALLTHQLQRLRIQPSSPPPNHPNNHLFRQPSNSPPPMSSAMIQPHGAASSSQFQGLPSRSAIFQQQPENCSSPPNVAL

- data["drugs"]에서 data["drugs"].values 로 바꿈, 일단 values 안하면 dictionary 길이가 protein의 경우는 49까지 나옴, 즉 뭔가 의도치않은 character들이 여러개 들어간다는 뜻

`-` Training을 위해 affinity를 텐서화

In [39]:
_affinity = torch.tensor(data["affinity"].values).reshape([-1,1])

`-` _drug와 _protein은 현재 word2int를 통해 char로 이루어진 sequence에서 int로 이루어진 sequence가 된 꼴이다. 이 int를 pytorch의 tensor로 바꾼다.

In [40]:
model_input_d = torch.tensor(_drugs, dtype=int)
model_input_p = torch.tensor(_proteins, dtype=int)

`-` drug, protein, affinity를 dim = 1에 대해서 concat을 하였다. 나중에 training할 때는 drug, protein, affinity의 길이를 기억해서 인덱싱으로 나눌 것이다.

In [41]:
model_input = torch.cat([model_input_d, model_input_p,_affinity], dim=1)

In [42]:
model_input

tensor([[ 3.0000, 23.0000,  3.0000,  ..., 20.0000, 13.0000,  5.0000],
        [ 3.0000,  3.0000,  8.0000,  ..., 21.0000, 21.0000,  5.0000],
        [ 3.0000,  3.0000,  0.0000,  ..., 21.0000, 21.0000,  5.0000],
        ...,
        [ 3.0000, 14.0000,  8.0000,  ..., 21.0000, 21.0000,  5.0000],
        [ 3.0000, 16.0000,  8.0000,  ...,  9.0000,  8.0000,  8.2007],
        [ 3.0000, 15.0000,  3.0000,  ..., 21.0000, 21.0000,  5.0000]])

`-` training DataLoader, test DataLoader  

In [43]:
train_loader = DataLoader(model_input[:round(25772 *0.8)], batch_size=256, shuffle=True , drop_last = True)
test_loader = DataLoader(model_input[round(25772 *0.8):], batch_size=256, shuffle=True, drop_last = True)

`-` 모델 파라메터 수 확인

In [197]:
count = 0
for parameter in model.parameters():
    if(parameter.dim() >= 2):
        # print(parameter.shape, np.prod(parameter.shape))
        count += np.prod(parameter.shape)
        
print(count)


torch.Size([37, 128]) 4736
torch.Size([49, 128]) 6272
torch.Size([32, 128, 4]) 16384
torch.Size([64, 32, 6]) 12288
torch.Size([96, 64, 8]) 49152
torch.Size([32, 128, 4]) 16384
torch.Size([64, 32, 8]) 16384
torch.Size([96, 64, 12]) 73728
torch.Size([1024, 192]) 196608
torch.Size([1024, 1024]) 1048576
torch.Size([512, 1024]) 524288
torch.Size([1, 512]) 512
1965312


`-`  내가 만든 모델에 내가 놓쳐서 넣지 않은 파트(AdaptiveMaxPool1d와 Conv 중간중간에 ReLU) 추가 + 다른 트레이닝 코드로 나온 결과 (내가 만든 트레이닝 코드가 아님)

In [77]:
class CNN(nn.Module):
    
    def __init__(self, drug_n_in_channel = 85, protein_n_in_channel = 1200 , n_filters = 32, drug_kernel_size = [8,8,8], protein_kernel_size = [8,8,8]):
        super().__init__()
        
        
        self.smiles_emb = nn.Embedding(25,128)
        self.protein_emb = nn.Embedding(25,128)
        
        self.smiles_Conv1 = nn.Conv1d(in_channels = 128, out_channels = n_filters, kernel_size = drug_kernel_size[0] )
        self.smiles_Conv2 = nn.Conv1d(in_channels = n_filters, out_channels = n_filters*2, kernel_size = drug_kernel_size[1] )
        self.smiles_Conv3 = nn.Conv1d(in_channels = n_filters*2, out_channels = n_filters*3, kernel_size = drug_kernel_size[2] )        
        self.smiles_MaxPool = nn.MaxPool1d(kernel_size = drug_kernel_size[0], stride = 1)
        self.smiles_AdaMaxPool = nn.AdaptiveMaxPool1d(1)   # 원래 내가 만든 model엔 없던 파트, 이거말고 MaxPool 사용했음

        
        self.protein_Conv1 = nn.Conv1d(in_channels = 128, out_channels = n_filters, kernel_size = protein_kernel_size[0] )
        self.protein_Conv2 = nn.Conv1d(in_channels = n_filters, out_channels = n_filters*2, kernel_size = protein_kernel_size[1] )
        self.protein_Conv3 = nn.Conv1d(in_channels = n_filters*2, out_channels = n_filters*3, kernel_size = protein_kernel_size[2] )
        self.protein_MaxPool = nn.MaxPool1d(kernel_size = protein_kernel_size[0], stride = 1 )
        self.protein_AdaMaxPool = nn.AdaptiveMaxPool1d(1)  # 원래 내가 만든 model엔 없던 파트, 이거말고 MaxPool 사용함
        
        self.linear1 = nn.Linear(192  ,1024, bias=True)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.1)
        
        self.linear2 = nn.Linear(1024, 1024, bias=True)
        self.dropout2 = nn.Dropout(0.1)

        self.linear3 = nn.Linear(1024, 512, bias=True)
        self.dropout3 = nn.Dropout(0.1)

        
        self.linear_final = nn.Linear(512, 1, bias=True)
        
        
    def forward(self, drug, protein):
        
       # print(drug.is_cuda)
       # print(protein.is_cuda)
        
        X = self.smiles_emb(drug)
        
        trans = X.permute(0,2,1)
        
        drug = self.smiles_Conv1(trans)
        drug = self.relu(drug) # 이전엔 추가 안했던 부분
        
        
        drug = self.smiles_Conv2(drug)
        drug = self.relu(drug) # 이전엔 추가 안했던 부분
        
        drug = self.smiles_Conv3(drug)
        drug = self.relu(drug) # 이전엔 추가 안했던 부분 
#        print("drug before maxpool:",drug.shape)
        
        drug = self.smiles_AdaMaxPool(drug) # 이전엔 그냥 MaxPool 했던 부분 
#        print("drug after maxpool:",drug.shape)        
        
        drug = drug.view(drug.shape[0],-1)        # 이전엔 추가 안했던 부분 

#        drug = torch.max(drug,dim=2).values 
#        protein = torch.max(protein,dim=2).values # MaxPooling하면 96개의 channel들에 대해서 Maxpooling이 되어서 이 96개 처리 어떻게 할지 https://kaya-dev.tistory.com/6 참고했음
  
        
        X = self.protein_emb(protein) 
        
        trans = X.permute(0,2,1)
        
        protein = self.protein_Conv1(trans)
        protein = self.relu(protein) # 이전엔 추가 안했던 부분        
        
        protein = self.protein_Conv2(protein)
        protein = self.relu(protein) # 이전엔 추가 안했던 부분        
        
        protein = self.protein_Conv3(protein)        
        protein = self.relu(protein) # 이전엔 추가 안했던 부분        

        protein = self.protein_AdaMaxPool(protein)  # 이전엔 그냥 MaxPool 했던 부분 
        
        protein = protein.view(protein.shape[0],-1)       # 이전엔 추가 안했던 부분 
        
       # print(drug.shape, protein.shape)#
                
        drug_protein = torch.cat([drug,protein],dim = 1) 
        
        drug_protein = self.linear1(drug_protein)
        drug_protein = self.relu(drug_protein)
        drug_protein = self.dropout1(drug_protein)
        
        drug_protein = self.linear2(drug_protein)
        drug_protein = self.relu(drug_protein)
        drug_protein = self.dropout2(drug_protein)

        drug_protein = self.linear3(drug_protein)
        drug_protein = self.relu(drug_protein)        
        
        #drug_protein = self.linear4(drug_protein)    
        
        drug_protein = self.linear_final(drug_protein)
        
        return drug_protein
    
        

- AdaptiveMaxPool1d 관련: https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveMaxPool1d.html

In [78]:
model = CNN()
model.to("cuda")

_protein_vocab

22

In [79]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [80]:
import time

In [81]:
def mse(y,f): #measure : mean squared error
    mse = ((y - f)**2).mean(axis=0)
    return mse

In [82]:
NUM_EPOCHS = 100

for epoch in range(NUM_EPOCHS):
    start = time.time()
    model.train()
    epoch_loss = 0
    for i,data in enumerate(train_loader):
        drugs,targets,labels = data[:,:85], data[:,85:1200+85], data[:,-1]
        drugs = drugs.int()
        targets = targets.int()
        drugs = drugs.to(device)
        targets = targets.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = model(drugs,targets)

        #print("Training:  output[0]: ", output[0].item(), "  labels[0]",labels[0].item())        

        loss = loss_fn(output, labels.view(-1,1).float().to(device))
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    train_loss = np.average(train_losses)

    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()

    with torch.no_grad():
        for data in test_loader:

            drugs,targets,labels = data[:,:85], data[:,85:1200+85], data[:,-1]
            drugs = drugs.int()
            targets = targets.int()
            drugs = drugs.to(device)
            targets = targets.to(device)
            labels = labels.to(device)                
            output = model(drugs,targets)
            
#            print("Validation:  output[0]: ", output[0].item(), "  labels[0]",labels[0].item())
            
            total_preds = torch.cat((total_preds, output.cpu()), 0)
            total_labels = torch.cat((total_labels, labels.view(-1,1).cpu()), 0)
    G,P = total_labels.numpy().flatten(),total_preds.numpy().flatten()
    
    
    

    # epoch당 평균 loss 계산
    train_loss = np.average(train_losses)
    test_loss = mse(G,P)

    epoch_len = len(str(NUM_EPOCHS))


    print_msg = (f'[{epoch}/{NUM_EPOCHS}] ' +
                 f'train_loss: {train_loss:.5f} ' +
                 f'test_loss: {test_loss:.5f}' +
                "The Time is:",time.time() - start)

    print(print_msg)

    # clear lists to track next epoch
    train_losses = []
    test_losses = []


    counter = 0 #patience
    best_mse = test_loss



('[0/100] train_loss: 1.05410 test_loss: 0.44974The Time is:', 5.72038197517395)
('[1/100] train_loss: 0.50288 test_loss: 0.41310The Time is:', 5.69394850730896)
('[2/100] train_loss: 0.49453 test_loss: 0.53387The Time is:', 5.676478862762451)
('[3/100] train_loss: 0.49655 test_loss: 0.43397The Time is:', 5.6628618240356445)
('[4/100] train_loss: 0.51443 test_loss: 0.52959The Time is:', 5.721963882446289)
('[5/100] train_loss: 0.50477 test_loss: 0.41002The Time is:', 5.677354097366333)
('[6/100] train_loss: 0.49810 test_loss: 0.40900The Time is:', 5.733128547668457)
('[7/100] train_loss: 0.53337 test_loss: 0.43228The Time is:', 5.75837779045105)
('[8/100] train_loss: 0.47489 test_loss: 0.41465The Time is:', 5.8811421394348145)
('[9/100] train_loss: 0.46325 test_loss: 0.39126The Time is:', 5.819016933441162)
('[10/100] train_loss: 0.45786 test_loss: 0.37501The Time is:', 5.7138283252716064)
('[11/100] train_loss: 0.43660 test_loss: 0.36046The Time is:', 5.719956636428833)
('[12/100] tra

KeyboardInterrupt: 

- 내가 만든 training 코드에서는 loss값의 단위가 논문과 맞기 않게 나와서 다른 코드를 사용

`-` 다른 분이 구현한 DeepDTA처럼 validation loss가 떨어지긴 했는데, 그 분은 100 epoch해서 loss 0.25가 나온 반면, 내 것은 34 epoch 정도에서 나왔었다.

# 모델 만드는 중에 마주친 문제 및 에러

- CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1. 
    - 에러 해결책 ->  https://mopipe.tistory.com/180
    
- CUDA관련 여러 에러 이슈들: https://nuggy875.tistory.com/135

1. 어떻게 word embeding으로 해서 각 단어에 대해 1dconv를 적용할지 -> 해결
    - https://happy-jihye.github.io/nlp/nlp-4/
    - https://wikidocs.net/80437
    - https://iamseungjun.tistory.com/15
    - https://kaya-dev.tistory.com/6 -> word embedding + MaxPooling
    
    <br/>
2. 256개의 데이터만 계속 보면 이 training set에 대해선 적어도 loss가 줄어야되는데 전혀 줄지가 않았다.
    - 원인은 모델 파라메터를 optimizer에 넘겨주는 코드를 실행하고 그 다음에 다시 모델 인스턴스를 만드는 순서로 갔기 때문이다. 반드시 모델을 선언한 다음에 optimizer에 모델 parameter 넘겨줘야한다.     
    
    
    <br/>    
3.  loss의 scale이 100 단위로 나오는 것이 잘 작동하는 것 같지 않다.
    - https://stackoverflow.com/questions/59153248/why-is-my-neural-network-stuck-at-high-loss-value-after-the-first-epochs
    - https://www.google.com/search?q=the+scale+of+loss+is+too+huge&oq=the+scale+of+loss+is+too+huge&aqs=chrome..69i57j33i160.4544j0j4&sourceid=chrome&ie=UTF-8

    - 일단 label shape가 data[:,-1]로 해야되는데 data[-1]로 했었다. 일단 이것을 data[:,-1] 로 바꿔서 1.0979899 까지 loss 내려간 것을 확인했다.
    
    - 다음은 output shape가 output.shape torch.Size([256, 96, 1])  인게 이상함. 이것도 256,1 꼴로 나와야될텐데, filter 처리를 제대로 못한 것 같음
    
    <br/>
4. embedding 벡터 shape 변환시키는 과정 시간이 너무 오래걸린다. -> 여러가지 고치고 나니까 해결되었다.


    <br/>
5. cuda를 데이터랑 모델에 했는데도 cuda, cpu안맞는다고 에러가 나왔는데 그 이유가, 내가 model class의 forward에서 embedding shape 바꾸려고 trans 텐서를 따로 선언한 것에 대해서 to cuda를 안한 것이 원인이였다.


    <br/>
6. 모델부터 training코드까지 잘 작성한 상태였고, training loss는 떨어지는 상황이였는데, validation loss만 떨어지지 않았었다.
    - 원인은 broadcasting으로 Drug와 Protein 데이터를 병합하고 [:0.5], [0.5:] 와 같은 방식으로 데이터를 나눠서, 모델이 사실상 앞의 0.5비율 만큼만 학습하고 뒤의 Drug는 전혀 학습을 못한 것이 원인이였던 것 같다. 이것을 shuffle하고 training을 다시 하니까 validation loss도 잘 감소하였다.